In [41]:
import datetime
import os
import re
import time
from urllib.parse import urlencode
from urllib.request import urlopen

import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from tqdm import tqdm

from src.constants._master import Master
from src.constants._results_cols import ResultsCols as Cols
from src.preparing.DataLoader import DataLoader
from src.preparing.prepare_chrome_driver import prepare_chrome_driver

In [3]:
from src.preparing.table_creator import TableCreator
from src.preparing.url_loader import KaisaiDateLoader
from tqdm import tqdm
import os
import pandas as pd
from bs4 import BeautifulSoup





def create_race_results_table_():
    race_html_list =["202001010101.bin","202001010102.bin", "202001010103.bin"]

    print("creating race_results_table")

    for race_html in tqdm(race_html_list):
        race_html_path = os.path.join("./data/html/race/", race_html)
        with open(race_html_path, "rb") as f:

            # 保存してあるbinファイルを読み込む
            html = f.read()
            # メインとなるレース結果テーブルデータを取得
            df = pd.read_html(html)
            print("df:",df)
            # htmlをsoupオブジェクトに変換
            soup = BeautifulSoup(html, "lxml")
            print("soup: ",soup)


alias = "race_results_table"
data_request = TableCreator()
data_request.set_args(alias)
data_request.pre_process_display()
create_race_results_table_()


race_results_table
self.from_location:  https://db.netkeiba.com/race/
to_temp_location:  ./data/tmp/race_results/
to_location:  ./data/html/race_results/
self.from_local_location:  ./data/html/race/
self.from_local_file_name:  *.bin
batch_size: 100
reloaded_*.bin type:  *.bin
race_results_table
self.from_location:  https://db.netkeiba.com/race/
to_temp_location:  ./data/tmp/race_results/
to_location:  ./data/html/race_results/
self.from_local_location:  ./data/html/race/
self.from_local_file_name:  *.bin
batch_size: 100
reloaded_*.bin type:  *.bin
creating race_results_table


100%|██████████| 3/3 [00:00<00:00, 16.13it/s]

df: [   着 順  枠 番  馬 番         馬名  性齢  斤量    騎手     タイム     着差    単勝  人 気      馬体重  \
0    1    6    6     ウインルーア  牝2  54  横山武史  1:49.7    NaN  16.0    3  438(+4)   
1    2    2    2     アークライト  牡2  54  ルメール  1:50.0  1.3/4   1.9    2   510(0)   
2    3    3    3  ギャラントウォリア  牡2  54  池添謙一  1:50.1    1/2   1.8    1  482(-6)   
3    4    1    1     ジュンブーケ  牝2  52  亀田温心  1:50.5  2.1/2  22.2    4   442(0)   
4    5    4    4    キタノマンゲツ  牡2  54  藤岡康太  1:51.0      3  55.7    5  426(-8)   
5    6    5    5     アイフレンズ  牝2  54   黛弘人  1:52.7      大  73.9    6  426(-2)   

        調教師  
0  [東] 青木孝文  
1  [東] 藤沢和雄  
2   [西] 平田修  
3   [西] 森秀行  
4  [西] 中尾秀正  
5  [東] 中野栄治  ,     0      1        2    3
0  単勝      6     1600    3
1  複勝    6 2  490 130  4 2
2  馬連  2 - 6     1410    4,      0                  1            2      3
0  ワイド  2 - 6 3 - 6 2 - 3  150 170 110  3 4 1
1   馬単              6 → 2         3920      9
2  三連複          2 - 3 - 6          240      2
3  三連単          6 → 2 → 3         5180    

In [23]:
#from src.preparing.table_creator import TableCreator
#from src.preparing.url_loader import KaisaiDateLoader
from tqdm import tqdm
import os
import pandas as pd
from bs4 import BeautifulSoup
import re
from src.constants._master import Master




def create_race_info_table_():
    race_html_list =["202001010101.bin","202001010102.bin", "202001010103.bin"]

    print("creating race_results_table")

    for race_html in tqdm(race_html_list):
        race_html_path = os.path.join("./data/html/race/", race_html)
        with open(race_html_path, "rb") as f:

            # 保存してあるbinファイルを読み込む
            html = f.read()
            #df = pd.read_html(html)[0]
            #print("df:",df)
            # htmlをsoupオブジェクトに変換
            soup = BeautifulSoup(html, "lxml")
            #print("soup: ",soup)
            # 天候、レースの種類、コースの長さ、馬場の状態、日付、回り、レースクラスをスクレイピング
            texts = (
                soup.find("div", attrs={"class": "data_intro"}).find_all("p")[0].text
                + soup.find("div", attrs={"class": "data_intro"}).find_all("p")[1].text
            )
            #print("texts: ",texts)
            info = re.findall(r"\w+", texts)
            #print("info: ",info)
            df = pd.DataFrame()

            # 障害レースフラグを初期化
            hurdle_race_flg = False
            for text in info:
                if text in ["芝", "ダート"]:
                    df["race_type"] = [text]
                if "障" in text:
                    df["race_type"] = ["障害"]
                    hurdle_race_flg = True
                if "m" in text:
                    # 20211212：[0]→[-1]に修正
                    df["course_len"] = [int(re.findall(r"\d+", text)[-1])]
                if text in Master.GROUND_STATE_LIST:
                    df["ground_state"] = [text]
                if text in Master.WEATHER_LIST:
                    df["weather"] = [text]
                if "年" in text:
                    df["date"] = [text]
                if "右" in text:
                    df["around"] = [Master.AROUND_LIST[0]]
                if "左" in text:
                    df["around"] = [Master.AROUND_LIST[1]]
                if "直線" in text:
                    df["around"] = [Master.AROUND_LIST[2]]
                if "新馬" in text:
                    df["race_class"] = [Master.RACE_CLASS_LIST[0]]
                if "未勝利" in text:
                    df["race_class"] = [Master.RACE_CLASS_LIST[1]]
                if ("1勝クラス" in text) or ("500万下" in text):
                    df["race_class"] = [Master.RACE_CLASS_LIST[2]]
                if ("2勝クラス" in text) or ("1000万下" in text):
                    df["race_class"] = [Master.RACE_CLASS_LIST[3]]
                if ("3勝クラス" in text) or ("1600万下" in text):
                    df["race_class"] = [Master.RACE_CLASS_LIST[4]]
                if "オープン" in text:
                    df["race_class"] = [Master.RACE_CLASS_LIST[5]]
            # グレードレース情報の取得
            grade_text = soup.find("div", attrs={"class": "data_intro"}).find_all("h1")[0].text
            if "G3" in grade_text:
                df["race_class"] = [Master.RACE_CLASS_LIST[6]] * len(df)
            elif "G2" in grade_text:
                df["race_class"] = [Master.RACE_CLASS_LIST[7]] * len(df)
            elif "G1" in grade_text:
                df["race_class"] = [Master.RACE_CLASS_LIST[8]] * len(df)

            # 障害レースの場合
            if hurdle_race_flg:
                df["around"] = [Master.AROUND_LIST[3]]
                df["race_class"] = [Master.RACE_CLASS_LIST[9]]

            # インデックスをrace_idにする
            race_id = re.findall(r'(\d+).bin',race_html)[0]
            df.index = [race_id] * len(df)
            race_infos= {}
            race_infos[race_id] = df
    race_infos_df = pd.concat([race_infos[key] for key in race_infos])
    print(race_infos_df)
#alias = "race_results_table"
#data_request = TableCreator()
#data_request.set_args(alias)
#data_request.pre_process_display()
create_race_info_table_()

creating race_results_table


100%|██████████| 3/3 [00:00<00:00, 44.08it/s]

              course_len around weather race_type ground_state        date  \
202001010103        1500      右       曇         芝            良  2020年7月25日   

             race_class  
202001010103        未勝利  


In [65]:
import time
from urllib.request import urlopen

def scrape_horse_id_list_():
    """
    当日出走するhorse_id一覧を取得
    """

    print("scraping horse_id_list")
    race_id_list =["202001010101","202001010102", "202001010103"]
    horse_id_list=[]
    for race_id in tqdm(race_id_list):
        # wait = WebDriverWait(driver, waiting_time)
        try:
            url = "https://db.netkeiba.com/race/"+ race_id

            # wait.until(EC.presence_of_element_located((By.CLASS_NAME, "HorseInfo")))
            html = urlopen(url)
            #soup = BeautifulSoup(html, "lxml", from_encoding="utf-8")
            soup = BeautifulSoup(html, "lxml")
            #print("soup",soup)
            horse_td_list = soup.find_all("td", attrs={"class": "txt_l"})
            horse_ids =[a['href'] for td in horse_td_list if (a := td.find('a')) and '/horse/' in a['href']]
            horse_id = [re.search(r'/horse/(\d+)/', href).group(1)  for href in horse_ids]
            horse_id_list.append(horse_id)
            # 内側のリストを外す
            flat_list = [item for sublist in horse_id_list for item in sublist]
            target_data = sorted(set(flat_list))
        except Exception as e:
            print(e)
            break
    print("horse_td_list:",target_data)



scrape_horse_id_list_()

scraping horse_id_list


100%|██████████| 3/3 [00:01<00:00,  2.49it/s]

horse_td_list: ['2017100432', '2017100813', '2017101364', '2017101757', '2017102190', '2017102915', '2017102918', '2017103355', '2017103622', '2017103709', '2017103841', '2017103915', '2017103953', '2017103996', '2017104065', '2017104160', '2017104334', '2017104792', '2017104859', '2017104882', '2017105136', '2017105153', '2017105242', '2017105261', '2017105324', '2017106065', '2017106103', '2017106389', '2018100828', '2018101626', '2018102410', '2018104800', '2018105193', '2018106434']


In [93]:
import datetime
import os
import re
import time
from urllib.parse import urlencode
from urllib.request import urlopen

import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from tqdm import tqdm

from src.preparing.DataLoader import DataLoader
from src.preparing.prepare_chrome_driver import prepare_chrome_driver

def scrape_peds_list_():
    """
    horse/pedページのhtmlを受け取って、血統のDataFrameに変換する関数。
    """
    # skip対象ではないゴミファイルの掃除

    ped_html_list = ["2018102657.bin"]#,"2017100282.bin"]
    data_index = 1
    peds = {}
    print("preparing peds table")
    for ped_html in tqdm(ped_html_list):
        ped_html_path = os.path.join("./data/html/ped/", ped_html)
        # print("ped_html_path", ped_html_path)
        try:
            with open(ped_html_path, "rb") as f:
                # 保存してあるbinファイルを読み込む
                html = f.read()
                # horse_idを取得
                #print("ped_html",ped_html)
                horse_id = re.findall(r"(\d+)", ped_html)[0]
                print("horse_id",horse_id)
                # htmlをsoupオブジェクトに変換
                soup = BeautifulSoup(html, "lxml")
                #print("soup",soup)
                peds_id_list = []
                # 血統データからhorse_idを取得する
                horse_a_list = soup.find("table", attrs={"summary": "5代血統表"}).find_all(
                    "a", attrs={"href": re.compile(r"^/horse/\w{10}")}
                )

                for a in horse_a_list:
                    # 血統データのhorse_idを抜き出す
                    work_peds_id = re.findall(r"horse\W(\w{10})", a["href"])[0]
                    print("work_peds_id",work_peds_id)
                    peds_id_list.append(work_peds_id)
                print("peds_id_list",peds_id_list)
                peds[horse_id] = peds_id_list
                print("peds",peds[horse_id] )
                # pd.DataFrame型にして一つのデータにまとめて、列と行の入れ替えして、列名をpeds_0, ..., peds_61にする
                peds_df = pd.DataFrame.from_dict(peds, orient="index").add_prefix("peds_")
                print("peds_df",peds_df)
            #if data_index % self.batch_size == 0:
            #    self.target_data = peds_df
            #    self.save_temp_file("peds_list")
            #    self.obtained_last_key = ped_html
        except Exception as e:
            print("Error at {}: {}".format(ped_html, e))
        #data_index = +1

scrape_peds_list_()

preparing peds table


100%|██████████| 1/1 [00:00<00:00, 17.78it/s]

horse_id 2018102657
work_peds_id 2012104511
work_peds_id 2001103460
work_peds_id 000a001d7e
work_peds_id 000a001607
work_peds_id 000a000e46
work_peds_id 000a007e0c
work_peds_id 000a009961
work_peds_id 000a001676
work_peds_id 000a0084c9
work_peds_id 000a00680e
work_peds_id 000a0001d6
work_peds_id 000a000378
work_peds_id 000a009241
work_peds_id 000a00b1e0
work_peds_id 000a000e1d
work_peds_id 000a009608
work_peds_id 2000107542
work_peds_id 000a00033a
work_peds_id 000a0012bf
work_peds_id 000a000f2b
work_peds_id 000a007459
work_peds_id 000a008c1e
work_peds_id 000a0019b6
work_peds_id 000a008c1d
work_peds_id 1993109154
work_peds_id 1983109006
work_peds_id 000a000827
work_peds_id 000a00859c
work_peds_id 1980101983
work_peds_id 000a000258
work_peds_id 1955105713
work_peds_id 2003101283
work_peds_id 000a000134
work_peds_id 000a001607
work_peds_id 000a000e46
work_peds_id 000a000f89
work_peds_id 000a007984
work_peds_id 000a007e0c
work_peds_id 000a000f87
work_peds_id 000a007d1c
work_peds_id 000a009

In [118]:


def create_horse_results_table_():
    """
    horse_htmlを受け取って、結果テーブルに変換する関数。data/html/horse_resultsに保存する

    """

    horse_html_list = ['2017103612.bin','2015105131.bin']
    data_index = 1
    print("creating horse_results_table")
    horse_results = {}
    horse_id_df = pd.DataFrame(columns=["horse_id"])
    for horse_html in tqdm(horse_html_list):
        #horse_html_path = os.path.join('./data/tmp/horse', horse_html)
        #with open(horse_html_path, "rb") as f:
        try:
            # 保存してあるbinファイルを読み込む
            #html = f.read
            # パスから正規表現でhorse_id_listを取得
            #print("horse_html",horse_html)
            #print("horse_html_path",horse_html_path)

            horse_id_list = re.findall(r"(\d+).bin", horse_html)
            #print("horse_id_list",horse_id_list)
            # DataFrameに追加
            horse_id_df = pd.concat([horse_id_df, pd.DataFrame({"horse_id": horse_id_list})], ignore_index=True)
            print("horse_id_df ", horse_id_df)

            ### 取得日マスタの更新 ###
            print("updating master")
            now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # 現在日時を取得

            pd.DataFrame(columns=["horse_id", "updated_at"])
            # マスタを読み込み
            master = pd.read_csv("horse_results.pickle", dtype=object)
            # horse_id列に新しい馬を追加
            #new_master = master.merge(horse_id_df, on="horse_id", how="outer")
            # マスタ更新
            #new_master.loc[new_master["horse_id"].isin(horse_id_list), "updated_at"] = now
            # 列が入れ替わってしまう場合があるので、修正しつつ保存
            #target_data = new_master[["horse_id", "updated_at"]]
            #obtained_last_key = horse_html
        except Exception as e:
            print("Error at {}: {}".format(horse_html_path, e))

        data_index += 1


create_horse_results_table_()


creating horse_results_table


100%|██████████| 2/2 [00:00<00:00, 453.49it/s]

horse_id_df       horse_id
0  2017103612
updating master
horse_id_df       horse_id
0  2017103612
1  2015105131
updating master


In [131]:
def create_horse_results_table_():
    #
    # horseページのhtmlを受け取って、馬の過去成績のDataFrameに変換する関数。
    horse_html_list = ['2017103612.bin']#,'2015105131.bin']
    data_index = 1
    horse_id_file_list = './data/tmp/horse'

    print("scraping horse_results_table")
    horse_results = {}
    df=pd.DataFrame()
    for horse_html in tqdm(horse_html_list):
        horse_html_path = os.path.join('./data/tmp/horse', horse_html)
        with open(horse_html_path, "rb") as f:
            try:
                # 保存してあるbinファイルを読み込む
                html = f.read()
                #print("html",html)
                df = pd.read_html(html)[3]
                print("df",df)
                # 受賞歴がある馬の場合、3番目に受賞歴テーブルが来るため、4番目のデータを取得する
                if df.columns[0] == "受賞歴":
                    df = pd.read_html(html)[4]
                    print("df4",df)
                # 新馬の競走馬レビューが付いた場合、
                # 列名に0が付与されるため、次のhtmlへ飛ばす
                if df.columns[0] == 0:
                    print("horse_results empty case1 {}".format(horse_html))
                    continue

                horse_id = re.findall(r"(\d+).bin", horse_html)[0]
                print("horse_id",horse_id)
                df.index = [horse_id] * len(df)
                horse_results[horse_id] = df

            # 競走データが無い場合（新馬）を飛ばす
            except IndexError:
                print("horse_results empty case2 {}".format(horse_html))
                continue

            except Exception as e:
                print("Error at {}: {}".format(horse_html, e))

        # pd.DataFrame型にして一つのデータにまとめる
        horse_results_df = pd.concat([horse_results[key] for key in horse_results])
        # 列名に半角スペースがあれば除去する
    target_data = horse_results_df.rename(columns=lambda x: x.replace(" ", ""))
    print("target",target_data)



create_horse_results_table_()

scraping horse_results_table


100%|██████████| 1/1 [00:00<00:00, 42.19it/s]

df            日付    開催 天 気  R   レース名  映 像  頭 数  枠 番  馬 番  オ ッ ズ  ...   着差  \
0  2020/08/30  2小倉6   晴  6  3歳未勝利  NaN   18    7   13   10.1  ...  3.7   
1  2020/08/15  2小倉1   晴  7  3歳未勝利  NaN   18    5    9   20.5  ...  0.2   
2  2020/07/12  4阪神4   晴  3  3歳未勝利  NaN   16    4    7   65.5  ...  2.5   
3  2020/04/18  2阪神7   曇  4  3歳未勝利  NaN   18    2    3   25.0  ...  3.1   
4  2020/03/20  1阪神7   晴  6  3歳未勝利  NaN   16    8   16   29.7  ...  1.5   
5  2020/02/29  1中京1   曇  4  3歳未勝利  NaN   17    3    6   17.1  ...  0.6   
6  2020/02/09  2京都4   曇  6   3歳新馬  NaN   16    4    8   20.1  ...  0.2   

   ﾀｲﾑ 指数           通過        ペース    上り      馬体重 厩舎 ｺﾒﾝﾄ  備考  勝ち馬 (2着馬)     賞金  
0      **    13-15-5-6  35.9-36.3  39.5  460(+4)     NaN NaN    タニノヨセミテ    NaN  
1      **    18-18-5-5  35.4-35.9  35.8  456(-4)     NaN NaN       チャイカ  130.0  
2      **  15-14-15-14  37.8-36.7  37.8  460(-4)     NaN NaN  タンジェリンムーン    NaN  
3      **        12-13  35.9-35.1  37.4  464(+2)     NaN NaN   ラルナブリラーレ    NaN  

In [88]:
def create_table_for_predict_():
    """
    scheduled_raceに保管したhtmlをスクレイピング。
    dateはyyyy/mm/ddの形式。
    """

    race_html_list = [
        "202410020110.bin",
        "202410020111.bin",
        "202405010511.bin",
        "202408020510.bin",
        "202410020109.bin",
        "202408020509.bin",
        "202405010510.bin",
        "202405010509.bin",
        "202408020511.bin",
    ]
    # print("race_html_list", race_html_list)

    df = pd.DataFrame()
    for race_html in tqdm(race_html_list):
        race_html_path = os.path.join("./data/html/scheduled_race/", race_html)
        with open(race_html_path, "rb") as f:
            # 保存してあるbinファイルを読み込む
            race_id = race_html[:12]
            html = f.read()
            df = pd.read_html(html)

            # htmlをsoupオブジェクトに変換
            soup = BeautifulSoup(html, "lxml")
            soup_span = soup.find_all("span")
            # メインのテーブルの取得
            for tr in soup.find_all("tr", class_="HorseList"):
                row = []
                for td in tr.find_all("td"):
                    if "HorseInfo" in td.get("class", []):
                        anchor = td.find("a")
                        if anchor:
                            href = anchor.get("href")
                            row.append(re.findall(r"horse/(\d*)", href)[0])
                    elif "Jockey" in td.get("class", []):
                        anchor = td.find("a")
                        if anchor:
                            href = anchor.get("href")
                            row.append(re.findall(r"jockey/result/recent/(\w*)", href)[0])
                    elif "Trainer" in td.get("class", []):
                        anchor = td.find("a")
                        if anchor:
                            href = anchor.get("href")
                            row.append(re.findall(r"trainer/result/recent/(\w*)", href)[0])
                    # elif "Txt_R Popular" in td.get("class", []):
                    #    span = td.find("span")
                    #    if span:
                    #        row.append(re.findall(r"(?<=>)\d+(\.\d+)?", span.text)[0])

                    row.append(td.text)
                print("row", row)
                # rowをDataFrameに変換し、dfに連結
                # DataFrameのリストを作成
                # 列のラベル
                columns = list(range(16))
                # DataFrameの作成
                df = pd.DataFrame([row], columns=columns)

                print("df", df)

                # レース結果テーブルと列を揃える
            df = df[[0, 1, 5, 6, 12, 13, 11, 3, 7, 9]]
            df.columns = [
                Cols.WAKUBAN,
                Cols.UMABAN,
                Cols.SEX_AGE,
                Cols.KINRYO,
                Cols.TANSHO_ODDS,
                Cols.POPULARITY,
                Cols.WEIGHT_AND_DIFF,
                "horse_id",
                "jockey_id",
                "trainer_id",
            ]
            df.index = [race_id] * len(df)

            print("df", df)


"""


            # レース情報の取得
            texts = soup.find_element(By.CLASS_NAME, "RaceList_Item02").text
            texts = re.findall(r"\w+", texts)
            # 障害レースフラグを初期化
            hurdle_race_flg = False
            for text in texts:
                if "m" in text:
                    # 20211212：[0]→[-1]に修正
                    df["course_len"] = [int(re.findall(r"\d+", text)[-1])] * len(df)
                if text in Master.WEATHER_LIST:
                    df["weather"] = [text] * len(df)
                if text in Master.GROUND_STATE_LIST:
                    df["ground_state"] = [text] * len(df)
                if "稍" in text:
                    df["ground_state"] = [Master.GROUND_STATE_LIST[1]] * len(df)
                if "不" in text:
                    df["ground_state"] = [Master.GROUND_STATE_LIST[3]] * len(df)
                if "芝" in text:
                    df["race_type"] = [list(Master.RACE_TYPE_DICT.values())[0]] * len(df)
                if "ダ" in text:
                    df["race_type"] = [list(Master.RACE_TYPE_DICT.values())[1]] * len(df)
                if "障" in text:
                    df["race_type"] = [list(Master.RACE_TYPE_DICT.values())[2]] * len(df)
                    hurdle_race_flg = True
                if "右" in text:
                    df["around"] = [Master.AROUND_LIST[0]] * len(df)
                if "左" in text:
                    df["around"] = [Master.AROUND_LIST[1]] * len(df)
                if "直線" in text:
                    df["around"] = [Master.AROUND_LIST[2]] * len(df)
                if "新馬" in text:
                    df["race_class"] = [Master.RACE_CLASS_LIST[0]] * len(df)
                if "未勝利" in text:
                    df["race_class"] = [Master.RACE_CLASS_LIST[1]] * len(df)
                if "１勝クラス" in text:
                    df["race_class"] = [Master.RACE_CLASS_LIST[2]] * len(df)
                if "２勝クラス" in text:
                    df["race_class"] = [Master.RACE_CLASS_LIST[3]] * len(df)
                if "３勝クラス" in text:
                    df["race_class"] = [Master.RACE_CLASS_LIST[4]] * len(df)
                if "オープン" in text:
                    df["race_class"] = [Master.RACE_CLASS_LIST[5]] * len(df)

            # グレードレース情報の取得
            if len(soup.find_elements(By.CLASS_NAME, "Icon_GradeType3")) > 0:
                df["race_class"] = [Master.RACE_CLASS_LIST[6]] * len(df)
            elif len(soup.find_elements(By.CLASS_NAME, "Icon_GradeType2")) > 0:
                df["race_class"] = [Master.RACE_CLASS_LIST[7]] * len(df)
            elif len(soup.find_elements(By.CLASS_NAME, "Icon_GradeType1")) > 0:
                df["race_class"] = [Master.RACE_CLASS_LIST[8]] * len(df)

            # 障害レースの場合
            if hurdle_race_flg:
                df["around"] = [Master.AROUND_LIST[3]] * len(df)
                df["race_class"] = [Master.RACE_CLASS_LIST[9]] * len(df)

            df["date"] = [date] * len(df)
        # except Exception as e:
        #    print(e)
        # finally:
        #    driver.close()
        #    driver.quit()
"""

create_table_for_predict_()

 22%|██▏       | 2/9 [00:00<00:00,  8.50it/s]

row ['', '', '\n\n--◎◯▲△☆&#10003消\n\n', '2018104534', '\n\n\nアスクキングコング\n\n\n', 'セ6', '55.0', '01174', '\n岩田望\n', '01084', '栗東梅田', '\n', '---.-', '\n**\n', '\n\n\n\n ', '']
df   0  1                        2           3                      4   5     6   \
0        \n\n--◎◯▲△☆&#10003消\n\n  2018104534  \n\n\nアスクキングコング\n\n\n  セ6  55.0   

      7        8      9     10  11     12      13         14 15  
0  01174  \n岩田望\n  01084  栗東梅田  \n  ---.-  \n**\n  \n\n\n\n      
row ['', '', '\n\n--◎◯▲△☆&#10003消\n\n', '2019104675', '\n\n\nアップストローク\n\n\n', 'セ5', '57.0', '01179', '\n菅原明\n', '01008', '美浦杉浦', '\n', '---.-', '\n**\n', '\n\n\n\n ', '']
df   0  1                        2           3                     4   5     6   \
0        \n\n--◎◯▲△☆&#10003消\n\n  2019104675  \n\n\nアップストローク\n\n\n  セ5  57.0   

      7        8      9     10  11     12      13         14 15  
0  01179  \n菅原明\n  01008  美浦杉浦  \n  ---.-  \n**\n  \n\n\n\n      
row ['', '', '\n\n--◎◯▲△☆&#10003消\n\n', '2020102971', '\n\n\nサト

 22%|██▏       | 2/9 [00:00<00:00,  7.26it/s]

row ['', '', '\n\n--◎◯▲△☆&#10003消\n\n', '2021105764', '\n\n\nアドマイヤベル\n\n\n', '牝3', '55.0', '\n\n○○\n\n', '01059', '美浦加藤征', '\n', '---.-', '\n**\n', '\n\n\n\n ', '']


ValueError: 16 columns passed, passed data had 15 columns